# Handling Missing Data

## What is missing data?

Not all missing data is equal. At the heart of the matter, there exists the need to distinguish between two types of missingness:

* **Unknown but existing data**: This is data that we know exists, however, due to sparse or incomplete sampling, we do not actually know the value of it. There is some value there, and it would be useful to try and apply some sort of missing data interpolation technique in order to discover it.

   For example, in 2013 *The New York Times* published [a survey of income mobility in the United States](http://www.nytimes.com/2013/07/22/business/in-climbing-income-ladder-location-matters.html). As it happens often in datasets which drill this deep (to a county level), there were several counties for which the newspaper could not trace data. Yet it would be possible, and easy, if it was truly necessary to do so, to interpolate reasonable values for these counties based on data from the surrounding ones, for instance, or based on data from other counties with similar demographic profiles. This is fundamentally speaking, data that *can* be filled by some means.

<!-- ![Map of the US by Income Mobility](location_matters.png "Map of the US by Income Mobility") -->
 
  
* **Data that doesn't exist**: data that does not exist at all, in any shape or form.

  For example, it would make no sense to ask the average household income for residents of an industrial park or other such location where no people actually live. It would not *really* make sense to use 0 as a [sentinal value](https://en.wikipedia.org/wiki/Sentinel_value) in this case, either, because the existance of such a number implies in the first place the existance of people for whom an average can be taken&mdash;otherwise in trying to compute an average you are making a [divide by zero error](https://en.wikipedia.org/wiki/Division_by_zero)! This is, fundamentally speaking, data that *cannot* be filled by any means.


This is an important distinction to keep in mind, and implementing it in some standard way significantly complicates the picture. It means that to ask the question "is this data entry filled?" one must actually consider three possible answers: "Yes", "No, but it can be", and "No, and it cannot be". There seem to be two dominant paradigms for handling this distinction:

* **Bitpatterns**: Embed sentinal values into the array itself. For instance for integer data one might take `0` or `-9999` to signal unknown but existant data. This requires no overhead but can be confusing and oftentimes robs you of values that you might otherwise want to use (like `0` or `-9999`).


* **Masks**: Use a seperate boolean array to "mask" the data whenever missing data needs to be represented. This requires making a second array and knowing when to apply it to the dataset, but is more robust.

[Numpy](http://www.numpy.org/) is the linear algebra and vectorized mathematical operation library which underpins the Python scientific programming stack, and its methodologies inform how everything else works. Numpy has masks: these are provided via the `numpy.ma` module. But it has no native bitpatterns! There is still no performant native bitpattern `NA` type available whatsoever.

The lack of a native `NA` type, as is the case in, say, R, is a **huge** problem for libraries, like [Pandas](http://pandas.pydata.org/), that should be able to efficiently handle large datasets.

Indeed, **Pandas does not use the `numpy.ma` mask**. Masks are simply non-performant above for the purposes of a library that is expected to be able to handle literally millions of entries entirely in-memory, as `pandas` does. Pandas instead defines and uses its own null value sentinels, particularly `NaN` (`np.nan`) for null numbers and `NaT` (a psuedo-native handled under-the-hood); and then allows you to apply your own `isnull()` mask to your dataset (more on that shortly). 


### ``None``: Pythonic missing data

The first sentinel value used by Pandas is ``None``, a Python singleton object that is often used for missing data in Python code.
Because it is a Python object, ``None`` cannot be used in any arbitrary NumPy/Pandas array, but only in arrays with data type ``'object'`` (i.e., arrays of Python objects):

In [1]:
import numpy as np
import pandas as pd

In [2]:
val = np.array([6, None, 7, 5, 6])
val

array([6, None, 7, 5, 6], dtype=object)

This ``dtype=object`` means that the best common type representation NumPy could infer for the contents of the array is that they are Python objects.
While this kind of object array is useful for some purposes, any operations on the data will be done at the Python level, with much more overhead than the typically fast operations seen for arrays with native types:

In [3]:
%timeit np.arange(1E5, dtype='object').sum()
print()

3.97 ms ± 46.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)



In [4]:
%timeit np.arange(1E5, dtype='int').sum()
print()

125 µs ± 4.26 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)



The use of Python objects in an array also means that if you perform aggregations like ``sum()`` or ``min()`` across an array with a ``None`` value, you will generally get an error:

### ``NaN``: Missing numerical data

The other missing data representation, ``NaN`` (acronym for *Not a Number*), is different; it is a special floating-point value recognized by all systems that use the standard IEEE floating-point representation:

In [5]:
val1 = np.array([1, np.nan, 7, 1, 8]) 
val1.dtype

dtype('float64')

Notice that NumPy prefered a native floating-point type for this array: this means that unlike the object array from before, this array supports fast operations pushed into compiled code.
You should be aware that ``NaN`` is a bit like data virus–it infects any other object it touches.
Regardless of the operation, the result of arithmetic with ``NaN`` will be another ``NaN``:

In [6]:
6 + np.nan

nan

In [7]:
7 *  np.nan

nan

NumPy does provide some special aggregations that will ignore these missing values:

In [8]:
np.nansum(val1)

17.0

Keep in mind that ``NaN`` is specifically a floating-point value; there is no equivalent NaN value for integers, strings, or other types.

### NaN and None in Pandas

``NaN`` and ``None`` both have their place, and Pandas is built to handle the two of them nearly interchangeably, converting between them where appropriate:

In [9]:
pd.Series([1, np.nan, 2, None])

0    1.0
1    NaN
2    2.0
3    NaN
dtype: float64

Notice that in addition to casting the integer array to floating point, Pandas automatically converts the ``None`` to a ``NaN`` value.
(Be aware that there is a proposal to add a native integer NA to Pandas in the future; as of this writing, it has not been included).

While this type of magic may feel a bit hackish compared to the more unified approach to NA values in domain-specific languages like R, the Pandas sentinel/casting approach works quite well in practice and in my experience only rarely causes issues.

The following table lists the upcasting conventions in Pandas when NA values are introduced:

|Typeclass     | Conversion When Storing NAs | NA Sentinel Value      |
|--------------|-----------------------------|------------------------|
| ``floating`` | No change                   | ``np.nan``             |
| ``object``   | No change                   | ``None`` or ``np.nan`` |
| ``integer``  | Cast to ``float64``         | ``np.nan``             |
| ``boolean``  | Cast to ``object``          | ``None`` or ``np.nan`` |

Keep in mind that in Pandas, string data is always stored with an ``object`` dtype.

## Operating on Null Values

As we have seen, Pandas treats ``None`` and ``NaN`` as essentially interchangeable for indicating missing or null values.
To facilitate this convention, there are several useful methods for detecting, removing, and replacing null values in Pandas data structures.
They are:

- ``isnull()``: Generate a boolean mask indicating missing values
- ``notnull()``: Opposite of ``isnull()``
- ``dropna()``: Return a filtered version of the data
- ``fillna()``: Return a copy of the data with missing values filled or imputed

We will conclude this section with a brief exploration and demonstration of these routines.

### Create dataframe with missing values

In [10]:
raw_data = {'first_name': ['Jason', np.nan, 'Tina', 'Jake', 'Amy'], 
        'last_name': ['Miller', np.nan, 'Ali', 'Milner', 'Cooze'], 
        'age': [42, np.nan, 36, 24, 73], 
        'sex': ['m', np.nan, 'f', 'm', 'f'], 
        'preTestScore': [4, np.nan, np.nan, 2, 3],
        'postTestScore': [25, np.nan, np.nan, 62, 70]}
df = pd.DataFrame(raw_data, columns = ['first_name', 'last_name', 'age', 'sex', 'preTestScore', 'postTestScore'])
df

,first_name,last_name,age,sex,preTestScore,postTestScore
0,Jason,Miller,42.0,m,4.0,25.0
1,NaN,NaN,NaN,NaN,NaN,NaN
2,Tina,Ali,36.0,f,NaN,NaN
3,Jake,Milner,24.0,m,2.0,62.0
4,Amy,Cooze,73.0,f,3.0,70.0


### Drop missing observations

In [11]:
df_no_missing = df.dropna()
df_no_missing

,first_name,last_name,age,sex,preTestScore,postTestScore
0,Jason,Miller,42.0,m,4.0,25.0
3,Jake,Milner,24.0,m,2.0,62.0
4,Amy,Cooze,73.0,f,3.0,70.0


### Drop rows where all cells in that row is NA

In [12]:
df_cleaned = df.dropna(how='all')
df_cleaned

,first_name,last_name,age,sex,preTestScore,postTestScore
0,Jason,Miller,42.0,m,4.0,25.0
2,Tina,Ali,36.0,f,NaN,NaN
3,Jake,Milner,24.0,m,2.0,62.0
4,Amy,Cooze,73.0,f,3.0,70.0


### Create a new column full of missing values

In [13]:
df['location'] = np.nan
df

,first_name,last_name,age,sex,preTestScore,postTestScore,location
0,Jason,Miller,42.0,m,4.0,25.0,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Tina,Ali,36.0,f,NaN,NaN,NaN
3,Jake,Milner,24.0,m,2.0,62.0,NaN
4,Amy,Cooze,73.0,f,3.0,70.0,NaN


### Drop column if they only contain missing values

In [14]:
df.dropna(axis=1, how='all')

,first_name,last_name,age,sex,preTestScore,postTestScore
0,Jason,Miller,42.0,m,4.0,25.0
1,NaN,NaN,NaN,NaN,NaN,NaN
2,Tina,Ali,36.0,f,NaN,NaN
3,Jake,Milner,24.0,m,2.0,62.0
4,Amy,Cooze,73.0,f,3.0,70.0


### Drop rows that contain less than five observations

This is mostly useful for time series

In [15]:
df.dropna(thresh=5)


,first_name,last_name,age,sex,preTestScore,postTestScore,location
0,Jason,Miller,42.0,m,4.0,25.0,NaN
3,Jake,Milner,24.0,m,2.0,62.0,NaN
4,Amy,Cooze,73.0,f,3.0,70.0,NaN


### Fill in missing data with zeros

In [16]:
df.fillna(0)

,first_name,last_name,age,sex,preTestScore,postTestScore,location
0,Jason,Miller,42.0,m,4.0,25.0,0.0
1,0,0,0.0,0,0.0,0.0,0.0
2,Tina,Ali,36.0,f,0.0,0.0,0.0
3,Jake,Milner,24.0,m,2.0,62.0,0.0
4,Amy,Cooze,73.0,f,3.0,70.0,0.0


### Fill in missing in preTestScore with the mean value of preTestScore

inplace=True means that the changes are saved to the df right away

In [17]:
df["preTestScore"].fillna(df["preTestScore"].mean(), inplace=True)
df

,first_name,last_name,age,sex,preTestScore,postTestScore,location
0,Jason,Miller,42.0,m,4.0,25.0,NaN
1,NaN,NaN,NaN,NaN,3.0,NaN,NaN
2,Tina,Ali,36.0,f,3.0,NaN,NaN
3,Jake,Milner,24.0,m,2.0,62.0,NaN
4,Amy,Cooze,73.0,f,3.0,70.0,NaN


### Fill in missing in postTestScore with each sex's mean value of postTestScore

In [18]:
df["postTestScore"].fillna(df.groupby("sex")["postTestScore"].transform("mean"), inplace=True)
df

,first_name,last_name,age,sex,preTestScore,postTestScore,location
0,Jason,Miller,42.0,m,4.0,25.0,NaN
1,NaN,NaN,NaN,NaN,3.0,NaN,NaN
2,Tina,Ali,36.0,f,3.0,70.0,NaN
3,Jake,Milner,24.0,m,2.0,62.0,NaN
4,Amy,Cooze,73.0,f,3.0,70.0,NaN


### Select some raws but ignore the missing data points

In [19]:
df[df['age'].notnull() & df['sex'].notnull()]

,first_name,last_name,age,sex,preTestScore,postTestScore,location
0,Jason,Miller,42.0,m,4.0,25.0,NaN
2,Tina,Ali,36.0,f,3.0,70.0,NaN
3,Jake,Milner,24.0,m,2.0,62.0,NaN
4,Amy,Cooze,73.0,f,3.0,70.0,NaN


In [20]:
# back-fill
df.fillna(method='bfill')

,first_name,last_name,age,sex,preTestScore,postTestScore,location
0,Jason,Miller,42.0,m,4.0,25.0,NaN
1,Tina,Ali,36.0,f,3.0,70.0,NaN
2,Tina,Ali,36.0,f,3.0,70.0,NaN
3,Jake,Milner,24.0,m,2.0,62.0,NaN
4,Amy,Cooze,73.0,f,3.0,70.0,NaN


In [21]:
df.fillna(method='ffill', axis=1)

,first_name,last_name,age,sex,preTestScore,postTestScore,location
0,Jason,Miller,42,m,4,25,25
1,NaN,NaN,NaN,NaN,3,3,3
2,Tina,Ali,36,f,3,70,70
3,Jake,Milner,24,m,2,62,62
4,Amy,Cooze,73,f,3,70,70


# Hierarchical Indexing

Up to this point we've been focused primarily on one-dimensional and two-dimensional data, stored in Pandas ``Series`` and ``DataFrame`` objects, respectively.
Often it is useful to go beyond this and store higher-dimensional data–that is, data indexed by more than one or two keys.
While Pandas does provide ``Panel`` and ``Panel4D`` objects that natively handle three-dimensional and four-dimensional data. This is a far more common pattern in practice is to make use of *hierarchical indexing* (also known as *multi-indexing*) to incorporate multiple index *levels* within a single index.
In this way, higher-dimensional data can be compactly represented within the familiar one-dimensional ``Series`` and two-dimensional ``DataFrame`` objects.

In this section, we'll traverse across the direct creation of ``MultiIndex`` objects, considerations when indexing, slicing, and computing statistics across multiple indexed data, and useful routines for conversions between simple and hierarchically indexed representations of your data.

We begin with the standard imports:

### Create dataframe

In [22]:
raw_data = {'regiment': ['Nighthawks', 'Nighthawks', 'Nighthawks', 'Nighthawks', 'Dragoons', 'Dragoons', 'Dragoons', 'Dragoons', 'Scouts', 'Scouts', 'Scouts', 'Scouts'], 
        'company': ['1st', '1st', '2nd', '2nd', '1st', '1st', '2nd', '2nd','1st', '1st', '2nd', '2nd'], 
        'name': ['Miller', 'Jacobson', 'Ali', 'Milner', 'Cooze', 'Jacon', 'Ryaner', 'Sone', 'Sloan', 'Piger', 'Riani', 'Ali'], 
        'preTestScore': [4, 24, 31, 2, 3, 4, 24, 31, 2, 3, 2, 3],
        'postTestScore': [25, 94, 57, 62, 70, 25, 94, 57, 62, 70, 62, 70]}
df = pd.DataFrame(raw_data, columns = ['regiment', 'company', 'name', 'preTestScore', 'postTestScore'])
df

,regiment,company,name,preTestScore,postTestScore
0,Nighthawks,1st,Miller,4,25
1,Nighthawks,1st,Jacobson,24,94
2,Nighthawks,2nd,Ali,31,57
3,Nighthawks,2nd,Milner,2,62
4,Dragoons,1st,Cooze,3,70
5,Dragoons,1st,Jacon,4,25
6,Dragoons,2nd,Ryaner,24,94
7,Dragoons,2nd,Sone,31,57
8,Scouts,1st,Sloan,2,62
9,Scouts,1st,Piger,3,70


### Set the hierarchical index but leave the columns inplace

In [23]:
df.set_index(['regiment', 'company'], drop=False)

regiment company      name  preTestScore  postTestScore
regiment   company                                                           
Nighthawks 1st      Nighthawks     1st    Miller             4             25
           1st      Nighthawks     1st  Jacobson            24             94
           2nd      Nighthawks     2nd       Ali            31             57
           2nd      Nighthawks     2nd    Milner             2             62
Dragoons   1st        Dragoons     1st     Cooze             3             70
           1st        Dragoons     1st     Jacon             4             25
           2nd        Dragoons     2nd    Ryaner            24             94
           2nd        Dragoons     2nd      Sone            31             57
Scouts     1st          Scouts     1st     Sloan             2             62
           1st          Scouts     1st     Piger             3             70
           2nd          Scouts     2nd     Riani             2             62
           2nd          Scouts     2nd       Ali             3             70

### Set the hierarchical index to be by regiment, and then by company

In [24]:
df.set_index(['regiment', 'company'], inplace = True)
df

name  preTestScore  postTestScore
regiment   company                                       
Nighthawks 1st        Miller             4             25
           1st      Jacobson            24             94
           2nd           Ali            31             57
           2nd        Milner             2             62
Dragoons   1st         Cooze             3             70
           1st         Jacon             4             25
           2nd        Ryaner            24             94
           2nd          Sone            31             57
Scouts     1st         Sloan             2             62
           1st         Piger             3             70
           2nd         Riani             2             62
           2nd           Ali             3             70

### View the index

In [25]:
df.index

MultiIndex(levels=[['Dragoons', 'Nighthawks', 'Scouts'], ['1st', '2nd']],
           labels=[[1, 1, 1, 1, 0, 0, 0, 0, 2, 2, 2, 2], [0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1]],
           names=['regiment', 'company'])

### Swap the levels in the index

In [26]:
df.swaplevel('regiment', 'company')

name  preTestScore  postTestScore
company regiment                                         
1st     Nighthawks    Miller             4             25
        Nighthawks  Jacobson            24             94
2nd     Nighthawks       Ali            31             57
        Nighthawks    Milner             2             62
1st     Dragoons       Cooze             3             70
        Dragoons       Jacon             4             25
2nd     Dragoons      Ryaner            24             94
        Dragoons        Sone            31             57
1st     Scouts         Sloan             2             62
        Scouts         Piger             3             70
2nd     Scouts         Riani             2             62
        Scouts           Ali             3             70

### Summarize the results by regiment

In [27]:
df.sum(level='regiment')

,preTestScore,postTestScore
regiment,,
Dragoons,62,246
Nighthawks,61,238
Scouts,10,264


Nevertheless, it is sometimes useful to explicitly create a ``MultiIndex``; we'll see a couple of these methods here.

### Explicit MultiIndex constructors

For more flexibility in how the index is constructed, you can instead use the class method constructors available in the ``pd.MultiIndex``.
For example, as we did before, you can construct the ``MultiIndex`` from simple list of arrays giving the index values within each level:

In [28]:
pd.MultiIndex.from_arrays([['a', 'a', 'b', 'b'], [1, 2, 1, 2]])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

You can construct it from a list of tuples giving the multiple index values of each point:

In [29]:
pd.MultiIndex.from_tuples([('a', 1), ('a', 2), ('b', 1), ('b', 2)])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

You can even construct it from a Cartesian product of single indices:

In [30]:
pd.MultiIndex.from_product([['a', 'b'], [1, 2]])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

Similarly, you can construct the ``MultiIndex`` directly using internal encoding by passing ``levels`` (a list of lists containing available index values for each level) and ``labels`` (a list of lists that reference these labels):

In [31]:
pd.MultiIndex(levels=[['a', 'b'], [1, 2]],
              labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

Any of these objects can be passed as the ``index`` argument when creating a ``Series`` or ``Dataframe``, or be passed to the ``reindex`` method of an existing ``Series`` or ``DataFrame``.

### MultiIndex for columns

In a ``DataFrame``, the rows and columns are completely symmetric, and just as the rows can have multiple levels of indices, the columns can have multiple levels as well.
Consider the following, which is a mock-up of some (somewhat realistic) medical data:

In [32]:
# hierarchical indices and columns
index = pd.MultiIndex.from_product([[2013, 2014], [1, 2]],
                                   names=['year', 'visit'])
columns = pd.MultiIndex.from_product([['Bob', 'Guido', 'Sue'], ['HR', 'Temp']],
                                     names=['subject', 'type'])

# mock some data
data = np.round(np.random.randn(4, 6), 1)
data[:, ::2] *= 10
data += 37

# create the DataFrame
health_data = pd.DataFrame(data, index=index, columns=columns)
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      54.0  37.8  46.0  37.2  27.0  36.2
     2      28.0  37.2  40.0  38.1  39.0  37.1
2014 1      42.0  38.0  36.0  36.5  29.0  37.3
     2      39.0  37.4  40.0  37.6  26.0  35.3

Here we see how multi-indexing for both rows and columns can come in *very* handy.
This is fundamentally four-dimensional data, where the dimensions are the subject, the measurement type, the year, and the visit number.
With this in place we can, for example, index the top-level column by the person's name and get a full ``DataFrame`` containing just that person's information:

In [33]:
health_data['Guido']

type          HR  Temp
year visit            
2013 1      46.0  37.2
     2      40.0  38.1
2014 1      36.0  36.5
     2      40.0  37.6

For complicated records containing multiple labeled measurements across multiple times for many subjects (people, countries, cities, etc.) use of hierarchical rows and columns can be extremely convenient!

### Multiply indexed DataFrames

A multiply indexed ``DataFrame`` behaves in a similar manner.
Consider our toy medical ``DataFrame`` from before:

In [34]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      54.0  37.8  46.0  37.2  27.0  36.2
     2      28.0  37.2  40.0  38.1  39.0  37.1
2014 1      42.0  38.0  36.0  36.5  29.0  37.3
     2      39.0  37.4  40.0  37.6  26.0  35.3

Remember that columns are primary in a ``DataFrame``, and the syntax used for multiply indexed ``Series`` applies to the columns.
For example, we can recover Guido's heart rate data with a simple operation:

In [35]:
health_data['Guido', 'HR']

year  visit
2013  1        46.0
      2        40.0
2014  1        36.0
      2        40.0
Name: (Guido, HR), dtype: float64

In [36]:
health_data.iloc[:2, :2]

subject      Bob      
type          HR  Temp
year visit            
2013 1      54.0  37.8
     2      28.0  37.2

These indexers provide an array-like view of the underlying two-dimensional data, but each individual index in ``loc`` or ``iloc`` can be passed a tuple of multiple indices. For example:

In [37]:
health_data.loc[:, ('Bob', 'HR')]

year  visit
2013  1        54.0
      2        28.0
2014  1        42.0
      2        39.0
Name: (Bob, HR), dtype: float64

## Rearranging Multi-Indices

One of the keys to working with multiple indexed data is knowing how to effectively transform the data.
There are a number of operations that will preserve all the information in the dataset, but rearrange it for the purposes of various computations.
We saw a brief example of this in the ``stack()`` and ``unstack()`` methods, but there are many more ways to finely control the rearrangement of data between hierarchical indices and columns, and we'll explore them here.

### Sorted and unsorted indices

Earlier, we briefly mentioned a caveat, but we should emphasize it more here.
*Many of the ``MultiIndex`` slicing operations will fail if the index is not sorted.*
Let's take a look at this here.

We'll start by creating some simple multiple indexed data where the indices are *not lexographically sorted*:

In [38]:
index = pd.MultiIndex.from_product([['a', 'c', 'b'], [1, 2]])
data = pd.Series(np.random.rand(6), index=index)
data.index.names = ['char', 'int']
data

char  int
a     1      0.299183
      2      0.868183
c     1      0.830519
      2      0.675910
b     1      0.486075
      2      0.756051
dtype: float64

If we try to take a partial slice of this index, it will result in an error:

In [39]:
try:
    data['a':'b']
except KeyError as e:
    print(type(e))
    print(e)

<class 'KeyError'>
'Key length (1) was greater than MultiIndex lexsort depth (0)'


Although it is not entirely clear from the error message, this is the result of the MultiIndex not being sorted.
For various reasons, partial slices and other similar operations require the levels in the ``MultiIndex`` to be sorted (i.e., lexographical) in order.
Pandas provides a number of convenience routines to perform this type of sorting; examples are the ``sort_index()`` and ``sortlevel()`` methods of the ``DataFrame``.
We'll use the simplest, ``sort_index()``, here:

In [40]:
data = data.sort_index()
data

char  int
a     1      0.299183
      2      0.868183
b     1      0.486075
      2      0.756051
c     1      0.830519
      2      0.675910
dtype: float64

With the index sorted in this way, partial slicing will work as expected:

In [41]:
data['a':'b']

char  int
a     1      0.299183
      2      0.868183
b     1      0.486075
      2      0.756051
dtype: float64